In [ ]:
!pip install ultralytics
!pip install -U ipywidgets
!pip install torch torchvision

In [ ]:
import torch
import pandas as pd

# Check if a GPU is available and move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# YOLO 알고리즘을 구현한 모델 다운로드
# COCO 데이터셋에서 학습된 YOLOv3 모델 사용
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# 이미지를 불러오고 전처리 수행
# PyTorch의 TorchVision 라이브러리를 사용하여 이미지를 불러오고, 이미지 크기를 조정하고, 채널 순서를 변경하여 모델의 입력 형식에 맞게 전처리를 수행
image = Image.open('/kaggle/input/pets-facial-expression-dataset/Master Folder/train/Angry/02.jpg')
transform = transforms.Compose([
    transforms.Resize((800, 800)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image = transform(image)

# 전처리된 이미지를 모델에 입력으로 넣고 객체 검출을 수행
model.eval()
with torch.no_grad():
    predictions = model([image])

# 예측된 bounding box들과 해당 객체의 클래스
bboxes = predictions[0]['boxes']
labels = predictions[0]['labels']

# 원본 이미지 위에 bounding box와 클래스 이름을 표시할 수 있습니다.
fig, ax = plt.subplots(1)
ax.imshow(image.permute(1, 2, 0))

for bbox, label in zip(bboxes, labels):
    ax.add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1], fill=False, edgecolor='red', linewidth=2))
    ax.text(bbox[0], bbox[1], f'{label.item()}', color='red', fontsize=12)

plt.show()

In [ ]:
import os
import torch
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# YOLOv8 모델 로드
model = YOLO('yolov8n.pt')  # Replace 'yolov8n.pt' with the appropriate model checkpoint

# 클래스 이름 가져오기
class_names = model.names

# 이미지 파일이 있는 디렉토리 경로 설정
image_dir = '/kaggle/input/pets-facial-expression-dataset/Master Folder/train/Angry/'  # 여기에 올바른 경로를 입력합니다
output_dir = 'annotated_images/'  # Annotated images will be saved here

# Annotated images 디렉토리가 존재하지 않으면 생성
os.makedirs(output_dir, exist_ok=True)

# 디렉토리 내의 모든 이미지 파일에 대해 반복
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # 이미지 파일 형식 필터링
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path)
        image_np = np.array(image)

        # YOLO 모델에 이미지를 입력으로 넣고 객체 검출 수행
        results = model(image_np)

        # 예측된 bounding box들과 해당 객체의 클래스
        pred = results[0]

        # 각 예측의 bounding box와 클래스 정보를 추출
        bboxes = pred.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
        confidences = pred.boxes.conf.cpu().numpy()  # confidence scores
        classes = pred.boxes.cls.cpu().numpy()  # class ids

        # 원본 이미지 위에 bounding box와 클래스 이름을 표시
        fig, ax = plt.subplots(1)
        ax.imshow(image)

        for bbox, conf, cls in zip(bboxes, confidences, classes):
            x1, y1, x2, y2 = bbox
            class_name = class_names[int(cls)]
            ax.add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2))
            ax.text(x1, y1, f'{class_name} {conf:.2f}', color='red', fontsize=12)

        # Annotated image 저장
        output_path = os.path.join(output_dir, filename)
        plt.savefig(output_path)
        plt.close()
        
        print(f'Processed and saved: {output_path}')

print('All images have been processed and saved.')

In [ ]:
import os
import torch
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# YOLOv8 모델 로드
model = YOLO('yolov8n.pt')  # Replace 'yolov8n.pt' with the appropriate model checkpoint

# 클래스 이름 가져오기
class_names = model.names

# 이미지 파일이 있는 디렉토리 경로 설정
image_dir = '/kaggle/input/pets-facial-expression-dataset/Master Folder/train/Angry/'  # 여기에 올바른 경로를 입력합니다
output_dir = 'annotated_images/'  # Annotated images will be saved here

# Annotated images 디렉토리가 존재하지 않으면 생성
os.makedirs(output_dir, exist_ok=True)

# 디렉토리 내의 모든 이미지 파일에 대해 반복
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # 이미지 파일 형식 필터링
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path)
        image_np = np.array(image)

        # YOLO 모델에 이미지를 입력으로 넣고 객체 검출 수행
        results = model(image_np)

        # 예측된 bounding box들과 해당 객체의 클래스
        pred = results[0]

        # 각 예측의 bounding box와 클래스 정보를 추출
        bboxes = pred.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
        confidences = pred.boxes.conf.cpu().numpy()  # confidence scores
        classes = pred.boxes.cls.cpu().numpy()  # class ids

        # 원본 이미지 위에 bounding box와 클래스 이름을 표시
        fig, ax = plt.subplots(1)
        ax.imshow(image)

        for bbox, conf, cls in zip(bboxes, confidences, classes):
            x1, y1, x2, y2 = bbox
            class_name = class_names[int(cls)]
            ax.add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2))
            ax.text(x1, y1, f'{class_name} {conf:.2f}', color='red', fontsize=12)

        # Annotated image 저장
        output_path = os.path.join(output_dir, filename)
        plt.savefig(output_path)
        plt.close()
        
        # Display the annotated image
        annotated_image = Image.open(output_path)
        plt.figure(figsize=(8, 8))
        plt.imshow(annotated_image)
        plt.axis('off')
        plt.title(f"Annotated Image: {filename}")
        plt.show()

        print(f'Processed and saved: {output_path}')

print('All images have been processed and saved.')

In [ ]:
import torch
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# YOLOv8 모델 로드
model = YOLO('yolov8n.pt')  # you can replace 'yolov8n.pt' with the appropriate model checkpoint

# 클래스 이름 가져오기
class_names = model.names

# 이미지를 불러오고 전처리 수행
image_path = '/kaggle/input/animals10/raw-img/cane/OIF-e2bexWrojgtQnAPPcUfOWQ.jpeg'  # 여기에 올바른 경로를 입력합니다
image = Image.open(image_path)
image = np.array(image)

# YOLO 모델에 이미지를 입력으로 넣고 객체 검출 수행
results = model(image)

# 예측된 bounding box들과 해당 객체의 클래스
# results[0]에는 예측된 결과가 포함됩니다.
pred = results[0]

# 각 예측의 bounding box와 클래스 정보를 추출
bboxes = pred.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
confidences = pred.boxes.conf.cpu().numpy()  # confidence scores
classes = pred.boxes.cls.cpu().numpy()  # class ids

# 원본 이미지 위에 bounding box와 클래스 이름을 표시
fig, ax = plt.subplots(1)
ax.imshow(image)

for bbox, conf, cls in zip(bboxes, confidences, classes):
    x1, y1, x2, y2 = bbox
    class_name = class_names[int(cls)]
    ax.add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2))
    ax.text(x1, y1, f'{class_name} {conf:.2f}', color='red', fontsize=12)

plt.show()

In [ ]:
# 데이터셋 경로
data_yaml_content = """
train: /kaggle/input/cat-dataset/AugmnetedCatsData.v1i.yolov8/AugmnetedCatsData.v1i.yolov8/train/images
val: /kaggle/input/cat-dataset/AugmnetedCatsData.v1i.yolov8/AugmnetedCatsData.v1i.yolov8/valid/images
test: /kaggle/input/cat-dataset/AugmnetedCatsData.v1i.yolov8/AugmnetedCatsData.v1i.yolov8/test/images

nc: 1  # 클래스 수 (고양이 얼굴만 탐지할 경우 1로 설정)
names: ['cat_face']  # 클래스 이름
"""

# YAML 파일 생성
yaml_file_path = '/kaggle/working/cat_data.yaml'
with open(yaml_file_path, 'w') as f:
    f.write(data_yaml_content)

print(f"YAML file created at: {yaml_file_path}")

In [ ]:
import os
import torch
from ultralytics import YOLO

# W&B 비활성화
os.environ['WANDB_DISABLED'] = 'true'

# YOLOv8 모델 로드
model = YOLO('yolov8n.pt')

# 데이터셋 경로 설정
data_yaml_path = '/kaggle/working/cat_data.yaml'

# 모델 학습
results = model.train(data=data_yaml_path, epochs=10, verbose=True)

**학습 로그**

Epoch: 현재 학습 중인 에폭(epoch) 번호입니다. 예를 들어, 1/10은 10 에폭 중 첫 번째 에폭을 의미합니다.

GPU_mem: GPU 메모리 사용량입니다. 각 에폭 동안 사용된 GPU 메모리 양을 나타냅니다.

box_loss: 박스 회귀 손실(Box regression loss)입니다. 모델이 예측한 박스와 실제 박스 간의 차이를 측정합니다. 값이 낮을수록 좋습니다.

cls_loss: 클래스 분류 손실(Classification loss)입니다. 모델이 객체의 클래스를 예측하는 데 발생하는 손실입니다. 값이 낮을수록 좋습니다.

dfl_loss: 디스트리뷰티드 폴리곤 손실(Distributed Focal Loss)입니다. 객체 검출의 정확도를 향상시키기 위한 손실입니다.

Instances: 한 배치에서 처리된 이미지의 수입니다.

Size: 입력 이미지의 크기입니다.

--------------

**검증 결과**

Box(P): 박스 예측의 정밀도(Precision)입니다. 모델이 예측한 박스가 실제 박스와 얼마나 일치하는지를 측정합니다.

Box(R): 박스 예측의 재현율(Recall)입니다. 실제 박스를 얼마나 잘 찾아냈는지를 측정합니다.

mAP50: IoU(Intersection over Union) 임계값 0.5에서의 평균 정밀도(Mean Average Precision)입니다. 높은 값일수록 좋습니다.

mAP50-95: IoU 임계값 0.5부터 0.95까지의 평균 정밀도입니다. 다양한 IoU 임계값에서 모델의 성능을 측정합니다.

--------------

**학습 결과 요약**

lr/pg0, lr/pg1, lr/pg2: 학습률(Learning Rate) 그래프입니다. 학습률의 변화를 시각화합니다.

metrics/mAP50(B): IoU 0.5에서의 평균 정밀도입니다. 0.995로 매우 높은 성능을 보여줍니다.

metrics/mAP50-95(B): IoU 0.5부터 0.95까지의 평균 정밀도입니다. 0.79로, 모델이 다양한 IoU 임계값에서 잘 동작함을 의미합니다.

metrics/precision(B): 객체 검출의 정밀도입니다. 0.9995로 매우 높은 정밀도를 보입니다.

metrics/recall(B): 객체 검출의 재현율입니다. 1.0으로, 모든 실제 객체를 성공적으로 검출했음을 의미합니다.

model/GFLOPs: 모델의 계산 복잡도를 나타내는 GFLOPs(Giga Floating Point Operations)입니다. 8.194 GFLOPs입니다.

model/parameters: 모델의 총 파라미터 수입니다. 3,010,043개입니다.

model/speed_PyTorch(ms): PyTorch에서의 모델 추론 속도입니다. 이미지당 5.324ms입니다.

train/box_loss, train/cls_loss, train/dfl_loss: 학습 데이터에 대한 손실 값입니다. 각 손실이 감소하는 것이 좋습니다.

val/box_loss, val/cls_loss, val/dfl_loss: 검증 데이터에 대한 손실 값입니다. 학습 손실과 검증 손실 간의 차이를 비교하여 모델의 일반화 능력을 평가할 수 있습니다.

결론
성능: 모델은 높은 정밀도(Precision)와 재현율(Recall)을 기록하며, mAP50과 mAP50-95가 매우 높아 객체 검출 성능이 우수함을 나타냅니다.
손실 감소: 학습 과정 동안 손실 값이 점차 감소하여 모델이 점점 더 좋은 성능을 보임을 의미합니다.
속도: 모델의 추론 속도가 적절하며, 파라미터 수가 상당하여 복잡한 모델임을 알 수 있습니다.

In [ ]:
test_path = '/kaggle/input/cat-dataset/AugmnetedCatsData.v1i.yolov8/AugmnetedCatsData.v1i.yolov8/test/images'

# 테스트 데이터에 대해 모델 예측 수행
test_results = model.predict(source=test_path, save=True)

# 결과 저장
model.save('yolov8_cat_detection_model.pt')  # 학습된 모델 저장

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('yolov8_cat_detection_model.pt')

# Path to the test image directory
test_image_dir = '/kaggle/input/cat-dataset/AugmnetedCatsData.v1i.yolov8/AugmnetedCatsData.v1i.yolov8/test/images'

# List files in the directory
image_files = os.listdir(test_image_dir)
print(f"Images in directory: {image_files}")

# 이미지 디렉토리 내 모든 이미지를 처리합니다.
for image_file in image_files:
    # 이미지 파일의 전체 경로
    image_path = os.path.join(test_image_dir, image_file)
    
    # 이미지 로드 및 RGB로 변환
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_file}")
        continue
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 이미지에 대한 예측 결과 가져오기
    results = model.predict(source=image_path)

    # 결과 리스트에서 현재 이미지에 대한 결과 가져오기
    result = results[0]

    # Bounding boxes, class labels, 그리고 confidence scores를 직접 가져오기
    boxes = result.boxes  # Boxes object
    if boxes is not None:
        for box in boxes:
            # x, y 좌표, 너비, 높이, confidence score, 클래스 ID를 추출합니다.
            x1, y1, x2, y2 = box.xyxy[0]  # Bounding box 좌표
            conf = box.conf[0]  # Confidence score
            cls = int(box.cls[0])  # Class ID
            label = f"{result.names[cls]} {conf:.2f}"

            # Bounding box 그리기
            cv2.rectangle(image_rgb, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            # 레이블 텍스트 추가
            cv2.putText(image_rgb, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 이미지 출력 (각 이미지별로 처리 결과를 확인하기 위해)
    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title(f"Predictions for {os.path.basename(image_path)}")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Load the trained YOLOv8 model
model = YOLO('yolov8_cat_detection_model.pt')

# 정확도 평가 (AP, Precision, Recall 등)
metrics = model.val(data=data_yaml_path)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Path to the image file
image_path = '/kaggle/working/runs/detect/train/F1_curve.png'

# Load and display the image
img = mpimg.imread(image_path)
plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.title('F1 Curve')
plt.show()

In [ ]:
import os
import torch
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# YOLOv8 모델 로드
model = YOLO('yolov8_cat_detection_model.pt')  # Replace 'yolov8n.pt' with the appropriate model checkpoint

# 클래스 이름 가져오기
class_names = model.names

# 이미지 파일이 있는 디렉토리 경로 설정
image_dir = '/kaggle/input/pets-facial-expression-dataset/Master Folder/train/Angry/'  # 여기에 올바른 경로를 입력합니다
output_dir = 'annotated_images/'  # Annotated images will be saved here

# Annotated images 디렉토리가 존재하지 않으면 생성
os.makedirs(output_dir, exist_ok=True)

# 디렉토리 내의 모든 이미지 파일에 대해 반복
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # 이미지 파일 형식 필터링
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path)
        image_np = np.array(image)

        # YOLO 모델에 이미지를 입력으로 넣고 객체 검출 수행
        results = model(image_np)

        # 예측된 bounding box들과 해당 객체의 클래스
        pred = results[0]

        # 각 예측의 bounding box와 클래스 정보를 추출
        bboxes = pred.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
        confidences = pred.boxes.conf.cpu().numpy()  # confidence scores
        classes = pred.boxes.cls.cpu().numpy()  # class ids

        # 원본 이미지 위에 bounding box와 클래스 이름을 표시
        fig, ax = plt.subplots(1)
        ax.imshow(image)

        for bbox, conf, cls in zip(bboxes, confidences, classes):
            x1, y1, x2, y2 = bbox
            class_name = class_names[int(cls)]
            ax.add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2))
            ax.text(x1, y1, f'{class_name} {conf:.2f}', color='red', fontsize=12)

        # Annotated image 저장
        output_path = os.path.join(output_dir, filename)
        plt.savefig(output_path)
        plt.close()
        
        # Display the annotated image
        annotated_image = Image.open(output_path)
        plt.figure(figsize=(8, 8))
        plt.imshow(annotated_image)
        plt.axis('off')
        plt.title(f"Annotated Image: {filename}")
        plt.show()

        print(f'Processed and saved: {output_path}')

print('All images have been processed and saved.')